In [ ]:

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

import numpy as np
import random
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
NUM_CLASSES = 20

#Class MLP


In [ ]:
class MLP:
  def __init__(self,vocab_size,hidden_size):
    self._vocab_size = vocab_size
    self._hidden_size = hidden_size
  def build_graph(self):
    self._X = tf.placeholder(tf.float32,shape= [None,self._vocab_size])
    self._real_Y = tf.placeholder(tf.int32,shape=[None,])
    weights_1 = tf.get_variable(
    name = 'weights_input_hidden',
    shape = (self._vocab_size,self._hidden_size),
    initializer = tf.random_normal_initializer(seed = 2021),
    )
    biases_1 = tf.get_variable(
        name = "biases_input_hidden",
        shape = (self._hidden_size),
        initializer = tf.random_normal_initializer(seed = 2021)
    )
    weights_2 = tf.get_variable(
    name = 'weights_hidden_output',
    shape = (self._hidden_size,NUM_CLASSES),
    initializer = tf.random_normal_initializer(seed = 2021),
    )
    biases_2 = tf.get_variable(
        name = "biases_hidden_output",
        shape = (NUM_CLASSES),
        initializer = tf.random_normal_initializer(seed = 2021)
    )
    hidden = tf.matmul(self._X,weights_1) + biases_1
    hidden = tf.sigmoid(hidden)
    logits = tf.matmul(hidden,weights_2) + biases_2

    labels_one_hot = tf.one_hot(indices=self._real_Y, depth=NUM_CLASSES, dtype = tf.float32)

    loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels_one_hot, logits = logits)

    loss = tf.reduce_mean(loss)

    probs = tf.nn.softmax(logits)

    predicted_labels = tf.argmax(probs,axis = 1)

    predicted_labels = tf.squeeze(predicted_labels)

    return predicted_labels, loss
  def trainer(self,loss,learning_rate):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_op





#DataReader

In [ ]:
class DataReader:
  def __init__(self,data_path,batch_size,vocab_size):
    self._batch_size = batch_size
    with open(data_path) as f:
      d_lines = f.read().splitlines()
    self._data = []
    self._labels = []
    for data_id, line in enumerate(d_lines):
      vector = [0.0 for _ in range(vocab_size)]
      features = line.split('<fff>')

      label, doc_id = int(features[0]), int(features[1])

      tokens = features[2].split()
      for token in tokens:
        index, value = int(token.split(":")[0]), float(token.split(":")[1])
        vector[index] = value
      self._data.append(vector)
      self._labels.append(label)

    self._data = np.array(self._data)
    self._labels = np.array(self._labels)

    
    self._num_epoch = 0
    self._batch_id = 0
  def next_batch(self):
    start = self._batch_id * self._batch_size
    end = start + self._batch_size
    self._batch_id += 1

    if (end + self._batch_size > len(self._data)):
      end = len(self._data)
      self._num_epoch  += 1
      self._batch_id = 0
      indices = range(len(self._data))
      random.seed(2021)
      random.shuffle(list(indices))
      tmpdata = []
      tmpy = []
      tmpdata = self._data[start:end]
      tmpy = self._labels[start:end]
      self._data, self._labels = self._data[indices], self._labels[indices]
      return tmpdata,tmpy
    return  self._data[start:end], self._labels[start:end]



In [ ]:
def load_dataset():
  train_data_reader = DataReader(
      data_path = "../datasets/20news-bydate/20news-train-tf-idf.txt",
      batch_size = 50,
      vocab_size = vocab_size
  )

  test_data_reader = DataReader(
      data_path = "../datasets/20news-bydate/20news-test-tf-idf.txt",
      batch_size = 50,
      vocab_size = vocab_size
  )
  
  return train_data_reader, test_data_reader

In [ ]:
def save_parameters(name,value,epoch):
  filename = name.replace(":","-colon-") + "-epoch-{}.txt".format(epoch)
  if len(value.shape) == 1:
    string_form = ','.join([str(number) for number in value])
  else:
    string_form = '\n'.join([','.join([str(number) for number in value[row]]) for row in range(value.shape[0])])
  with open("saved-paras/"+filename,'w') as f:
    f.write(string_form)

In [ ]:
def restore_parameters(name,epoch):
  filename = name.replace(":","-colon-") + "-epoch-{}.txt".format(epoch)
  with open("../saved-paras/"+filename) as f:
    lines = f.read().splitlines()
  if len(lines) == 1:
    value = [float(number) for number in lines[0].split(',')]
  else:
    value = [[float(number) for number in lines[row].split(',')] for row in range(len(lines))]
  return value

In [ ]:
with open("../datasets/20news-bydate/words_idfs.txt") as f:
  vocab_size = len(f.read().splitlines())
tf.reset_default_graph()
mlp = MLP(
    vocab_size = vocab_size,
    hidden_size = 50
)
predicted_labels, loss = mlp.build_graph()
train_op = mlp.trainer(loss = loss , learning_rate = 0.1)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
with tf.Session() as sess:
  train_data_reader,test_data_reader = load_dataset()
  print(train_data_reader._labels)
  step , MAX_STEP = 0, 10000
  tf.global_variables_initializer()

  sess.run(tf.global_variables_initializer())

  while step < MAX_STEP:

    train_data, train_labels =  train_data_reader.next_batch()
    
    plabel_eval, loss_eval, _ = sess.run(
        [predicted_labels, loss, train_op],
        feed_dict = {
            mlp._X : train_data,
            mlp._real_Y: train_labels
        }
    )
    step += 1
    print('step: {}, loss: {}'.format(step,loss_eval))

  trainable_variables = tf.trainable_variables()
  for variable in trainable_variables:
    save_parameters(
      name = variable.name,
      value = variable.eval(),
      epoch = train_data_reader._num_epoch
    )


Streaming output truncated to the last 5000 lines.
step: 5001, loss: 0.02486594021320343
step: 5002, loss: 0.00012954736303072423
step: 5003, loss: 4.7191941121127456e-05
step: 5004, loss: 5.5777512898202986e-05
step: 5005, loss: 9.724700794322416e-05
step: 5006, loss: 0.00018805851868819445
step: 5007, loss: 0.05018411576747894
step: 5008, loss: 0.0004954746109433472
step: 5009, loss: 0.0004638504469767213
step: 5010, loss: 0.0001632727071410045
step: 5011, loss: 0.0004336951533332467
step: 5012, loss: 0.00021524236944969743
step: 5013, loss: 0.0007000361219979823
step: 5014, loss: 0.00018228247063234448
step: 5015, loss: 0.04800159111618996
step: 5016, loss: 0.021962208673357964
step: 5017, loss: 0.00010516178735997528
step: 5018, loss: 5.547820546780713e-05
step: 5019, loss: 3.832156289718114e-05
step: 5020, loss: 8.2413200289011e-05
step: 5021, loss: 7.302707672351971e-05
step: 5022, loss: 2.7318579668644816e-05
step: 5023, loss: 0.0012954456033185124
step: 5024, loss: 0.0001162420

#Lưu tham số mô hình

#Khôi phục

In [ ]:
with tf.Session() as sess:
  trainable_variables = tf.trainable_variables()
  for variable in trainable_variables:
    saved_value = restore_parameters(variable.name, 44)
    assign_op = variable.assign(saved_value)
    sess.run(assign_op)